In [1]:
%load_ext autoreload
%load_ext autotime

time: 373 µs (started: 2023-03-02 14:01:10 -08:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

time: 719 µs (started: 2023-03-02 14:01:12 -08:00)


Some logs are stored in VAST with some stored in GPFS.

Set `log_dir` to `/vast1/` when running the notebook on Quartz, otherwise use `/gpfs1/`.

In [3]:
log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"


time: 1.94 ms (started: 2023-03-02 14:01:34 -08:00)


`RecorderAnalyzer` is the main part that initializes the cluster. It creates `LocalCluster` or `LSFCluster` depending on the `force_local` variable. When it is set to `true`, it will just create a `LocalCluster`. 

In [16]:
%autoreload 2

from wisio.recorder import VIEW_TYPES, RecorderAnalyzer

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=4,
            dashboard_port=3646,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
)

recorder_analyzer


time: 3.1 s (started: 2023-03-02 14:04:51 -08:00)


In [17]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]


['http://192.168.134.14:42439/status']

time: 11.7 ms (started: 2023-03-02 14:04:55 -08:00)


From this point on, every step is exactly the same within the `analyze_parquet` method of `RecorderAnalyzer`.

In [18]:
%autoreload 2

global_min_max = recorder_analyzer.load_global_min_max(log_dir=log_dir)
global_min_max


{'file_id': [901974072163427748, 4531734248727221027],
 'proc_id': [6860131432589748884, 6860131432673569428],
 'tmid': [0, 6683055996]}

time: 11.6 ms (started: 2023-03-02 14:04:55 -08:00)


In [22]:
%autoreload 2

from wisio._recorder.analysis import (
    compute_main_view,
    compute_max_io_time,
    compute_unique_file_names,
    compute_unique_proc_names,
    compute_view
)

time: 882 µs (started: 2023-03-02 14:05:26 -08:00)


`compute_main_view` automatically does `persist` before returning the DataFrame. You may play with that part if the `main_view` doesn't fit within the allocated memory. That said, `persist`ing it increases performance significantly.

In [28]:
%autoreload 2

main_view = compute_main_view(
    log_dir=log_dir,
    global_min_max=global_min_max,
    view_types=VIEW_TYPES
)

main_view


,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,func_id,read_time,write_time,metadata_time,read_size,write_size,metadata_size,read_count,write_count,metadata_count,data_count,data_size,data_time,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int64,int64,int64,float32,int64,int64,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 739 ms (started: 2023-03-02 14:08:04 -08:00)


`max_io_time` is just a `dd.Scalar` since I tried to compute everyting lazily.

In [29]:
%autoreload 2

max_io_time = compute_max_io_time(main_view=main_view)
max_io_time

dd.Scalar<series-..., dtype=float32>

time: 22.8 ms (started: 2023-03-02 14:08:11 -08:00)


`unique_file_names` and `unique_proc_names` are evaluated lazily too. I'm merging them back after computing `low_level_view`, `mid_level_view` and `high_level_view`.

In [30]:
%autoreload 2

unique_file_names = compute_unique_file_names(log_dir=log_dir)
unique_proc_names = compute_unique_proc_names(log_dir=log_dir)

time: 86.3 ms (started: 2023-03-02 14:08:14 -08:00)


You may configure `delta` and `cut` using the following variables.

In [26]:
DELTA = 0.0001
CUT = 0.5

time: 831 µs (started: 2023-03-02 14:06:50 -08:00)


The following is how all the views (including `expanded_view` and `bottleneck_view`-I didn't like the `cut_view` name). So, if you want to play with that part `compute_view` is the method you should look for.

In [31]:
%autoreload 2

import itertools as it

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Compute view
    views[view_permutation] = compute_view(
        main_view=main_view,
        views=views,
        view_permutation=view_permutation,
        max_io_time=max_io_time,
        delta=DELTA,
        cut=CUT
    )


len(views)

15

time: 2.63 s (started: 2023-03-02 14:08:19 -08:00)


The following just initializes the bottleneck detector.

In [32]:
%autoreload 2

from wisio._recorder.bottlenecks import RecorderBottleneckDetector

bottleneck_detector = RecorderBottleneckDetector(
    logger=recorder_analyzer.logger,
    log_dir=log_dir,
    views=views,
    view_types=VIEW_TYPES,
    unique_file_names=unique_file_names,
    unique_proc_names=unique_proc_names
)

bottleneck_detector


time: 4.95 ms (started: 2023-03-02 14:08:26 -08:00)


This part is where our nested loops lie. The output type is `{view_type: bottleneck_dict}`.

In [33]:
%autoreload 2

bottlenecks = bottleneck_detector.detect_bottlenecks(max_io_time=max_io_time)

len(bottlenecks)

15

time: 40.1 s (started: 2023-03-02 14:08:35 -08:00)


The following saves the bottlenecks per `view_type`. 

In [36]:
%autoreload 2

recorder_analyzer.save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)


time: 6.68 s (started: 2023-03-02 14:11:17 -08:00)
